Separately run referrals and claims queries.
This file imports the results of those queries, cleans and standardizes each, and merges them for analysis.

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import pyodbc
import os

## Import Data

In [2]:
# with open('referrals.sql', 'r') as myfile:
#     cpts_sql_str=myfile.read().replace('\n', ' ')

# cnxn_cpts = pyodbc.connect('DRIVER={SQL Server};SERVER=colo-dwrpt01;DATABASE=IADS_V3')

# cpts = pd.read_sql(cpts_sql_str, cnxn_cpts)

# cnxn_cpts.close()

In [3]:
# cpts.to_csv('../Data/cpts_raw.csv', sep='|')

In [4]:
# with open('claims.sql', 'r') as myfile:
#     claims_sql_str=myfile.read().replace('\n', ' ')

# cnxn_claims = pyodbc.connect('DRIVER={SQL Server};SERVER=colo-dwrpt01;DATABASE=NATIONAL_ANALYTICS')

# claims = pd.read_sql(claims_sql_str, cnxn_claims)

# cnxn_claims.close()

In [5]:
# claims.to_csv('../Data/claims_raw.csv', sep='|')

In [6]:
# with open('referrals_new.sql', 'r') as myfile:
#     cpts_new_sql_str=myfile.read().replace('\n', ' ')

# cnxn_cpts_new = pyodbc.connect('DRIVER={SQL Server};SERVER=colo-dwrpt01;DATABASE=IADS_V3')

# cpts_new = pd.read_sql(cpts_new_sql_str, cnxn_cpts_new)

# cnxn_cpts_new.close()

In [7]:
# cpts_new.to_csv('../Data/cpts_new_raw.csv', sep='|')

In [8]:
# with open('cpt_desc.sql', 'r') as myfile:
#     cpt_desc_sql_str=myfile.read().replace('\n', ' ')

# cnxn_cpt_desc = pyodbc.connect('DRIVER={SQL Server};SERVER=colo-dwrpt01;DATABASE=IADS_V3')

# cpt_desc = pd.read_sql(cpt_desc_sql_str, cnxn_cpt_desc)

# cnxn_cpt_desc.close()

In [9]:
# cpt_desc.to_csv('../Data/cpt_desc_raw.csv', sep='|')

In [10]:
clinical_decisions_20190503_remove = pd.read_csv('../data/clinical_decision_20190503_remove.csv')

In [11]:
clinical_decisions_20190503_add = pd.read_csv('../data/clinical_decision_20190503_add.csv')

In [12]:
clinical_decisions_20190517 = pd.read_csv('../data/clinical_decision_20190517.csv')

In [13]:
clinical_decisions_20190522 = pd.read_csv('../data/clinical_decision_20190522.csv')

In [14]:
cpts = pd.read_csv('../Data/cpts_raw.csv', sep='|')

In [15]:
cpts.drop(['Unnamed: 0'], inplace=True, axis=1)

In [16]:
claims = pd.read_csv('../Data/claims_raw.csv', sep='|')

In [17]:
claims.drop(['Unnamed: 0'], inplace=True, axis=1)

In [18]:
cpts_new = pd.read_csv('../Data/cpts_new_raw.csv', sep='|')

In [19]:
cpts_new.drop(['Unnamed: 0'], inplace=True, axis=1)

In [20]:
cpt_desc = pd.read_csv('../Data/cpt_desc_raw.csv', sep='|')

In [21]:
cpt_desc.drop(['Unnamed: 0'], inplace=True, axis=1)

## Clean cpts

In [22]:
list_o_specs_original = list(set(cpts['Specialty'].unique().tolist() + cpts_new['Specialty'].unique().tolist()))

In [23]:
cpts_head = cpts.drop_duplicates(subset='HCP_CONNECT_AUTH_NUMBER').reset_index()

cpts_head.drop(columns = ['index', 'CPT_Code', 'Date_Decision',
       'Date_Received', 'ref_type', 'region', 'UNITS'], inplace=True)

cpts_head = cpts_head[cpts_head['HCP_CONNECT_AUTH_NUMBER'].isna()==False].reset_index()

In [24]:
cpts['Date_Decision'] = pd.to_datetime(cpts['Date_Decision'])

In [25]:
cpts['Date_Received'] = pd.to_datetime(cpts['Date_Received'])

In [26]:
cpts['LOB'].isna().sum()

0

In [27]:
new_lob = {'COMMERCIAL': '_b',
               'SENIOR': '_s',
          'MEDI-CAL': '_s'}
cpts['LOB'] = cpts['LOB'].replace(new_lob)

In [28]:
## Define a list of specialties that will be broken out into LOB for the purposes of AA
specs_w_lob_distinct = ['RADIOLOGY']

In [29]:
for spec in list_o_specs_original:
    if spec in specs_w_lob_distinct:
        cpts['Specialty'] = np.where(cpts['Specialty']==spec, cpts['Specialty']+cpts['LOB'], cpts['Specialty'])
    

## Clean Claims

In [30]:
## Claims sum is used for the cost of a cpt code when it doesn't exist in claims for the associated specialty
claims_sum = claims.groupby(['CPT_Code'], as_index=False).agg({'avg_hcp_cost': 'mean'})

## Clean Clinical Decisions

In [31]:
clinical_decisions_20190503_remove['Notes'].unique()

array(['Concern for Over-Utilization',
       'Cost  containment with this speciality', 'GUIDELINE REQ',
       'Inappropriate location', 'NEEDS BENEFIT CHECK', 'Needs Review',
       'needs review', 'Not CMS benefit',
       'RNL Stauts, intentionally surpressed, not currently reviewed; additional discussion',
       'Unclassified Drugs', ' Capped Providers/Employed ', 'drug?',
       'Facility Code', 'Heavy cap volume',
       'ok to AA to employed/cap providers. R1,II,III,VI', 'Old code',
       'UPCODING', nan], dtype=object)

In [32]:
new_reason = {' Capped Providers/Employed ': 'capped providers',
               'Concern for Over-Utilization': 'overutilization concern',
               'Cost  containment with this speciality': 'cost containment',
               'drug?': 'UD',
               'Facility Code': 'facility code',
               'GUIDELINE REQ': 'guideline req', 
               'Heavy cap volume': 'heavy cap vol', 
               'Inappropriate location': 'inappropriate loc',
               'NEEDS BENEFIT CHECK': 'bene check', 
               'Needs Review': 'needs review', 
               'needs review': 'needs review',
               'Not CMS benefit': 'not CMS bene',
               'ok to AA to employed/cap providers. R1,II,III,VI': 'can approve',
               'Old code': 'old code',
               'RNL Stauts, intentionally surpressed, not currently reviewed; additional discussion': 'RNL',
               'Unclassified Drugs': 'unclassified drug', 
               'UPCODING': 'upcode'}
clinical_decisions_20190503_remove['Notes'] = clinical_decisions_20190503_remove['Notes'].replace(new_reason)
clinical_decisions_20190503_add['Notes'] = clinical_decisions_20190503_add['Notes'].replace(new_reason)

## Feature Engineering

In [33]:
## flag retro statuses with 1 and 0
retro_conditions = [
 (cpts['status_name'] == 'APPROVED - RETRO REVIEW') |
 (cpts['status_name'] == 'DENIED - RETRO REVIEW') |
 (cpts['status_name'] == 'APPROVED - COB RETRO') |
 (cpts['status_name'] == 'PENDING - RETRO REVIEW') 
  ]

In [34]:
## Create Refs - the older set of referrals from which we make the rules

In [35]:
choices = [1]
cpts['is_retro'] = np.select(retro_conditions, choices, default=0)

In [36]:
## remove retros from list and drop 'is_retro' as it is no longer needed
cpts = cpts[cpts['is_retro']==0]

cpts.drop(columns='is_retro', inplace=True)

In [37]:
cpts_det = cpts

cpts_det.drop(columns = ['PPL', 'Date_Decision',
       'Date_Received', 'ref_type', 'status_cat', 'status_name', 'Specialty',
       'region', 'UNITS'])

cpts_det = cpts_det[cpts_det['HCP_CONNECT_AUTH_NUMBER'].isna()==False].reset_index()

In [38]:
cpts['is_autoapp'] = np.where(cpts['status_name']=='APPROVED - AUTO', 1, 0)

In [39]:
cpts.PPL.fillna("N", inplace=True)

In [40]:
cpts['is_PPL'] = np.where(cpts['PPL']=='Y', 1, 0)

In [41]:
den_conditions = [
 (cpts['status_name'] == 'DENIED - CM') |
 (cpts['status_name'] == 'DENIED - BENEFIT CARVE OUT') |
 (cpts['status_name'] == 'DENIED - NOT A COVERED BENEFIT') |
 (cpts['status_name'] == 'DENIED - APPEAL') |
 (cpts['status_name'] == 'DENIED - CLINICAL TRIAL/EXP/INV') |
 (cpts['status_name'] == 'DENIED - TRANSPLANT') |
 (cpts['status_name'] == 'DENIED - MD') |
 (cpts['status_name'] == 'DENIED - CM/MD') |
 (cpts['status_name'] == 'DENIED - REDIRECT OSVN') |
 (cpts['status_name'] == 'DENIED - TICKLER')
  ]

In [42]:
choices = [1]
cpts['is_den'] = np.select(den_conditions, choices, default=0)

In [43]:
cpts['is_app'] = np.where(cpts['status_cat']=='APPROVED', 1, 0)

## Clean cpts_new - a new set of referrals to apply the rules to.

In [44]:
cpts_new['Date_Decision'] = pd.to_datetime(cpts_new['Date_Decision'])

In [45]:
cpts_new['Date_Received'] = pd.to_datetime(cpts_new['Date_Received'])

In [46]:
new_lob = {'COMMERCIAL': '_b',
               'SENIOR': '_s',
          'MEDI-CAL': '_s'}
cpts_new['LOB'] = cpts_new['LOB'].replace(new_lob)

In [47]:
for spec in list_o_specs_original:
    if spec in specs_w_lob_distinct:
        cpts_new['Specialty'] = np.where(cpts_new['Specialty']==spec, cpts_new['Specialty']+cpts_new['LOB'], cpts_new['Specialty'])
    

In [48]:
## flag retro statuses with 1 and 0
retro_conditions_new = [
 (cpts_new['status_name'] == 'APPROVED - RETRO REVIEW') |
 (cpts_new['status_name'] == 'DENIED - RETRO REVIEW') |
 (cpts_new['status_name'] == 'APPROVED - COB RETRO') |
 (cpts_new['status_name'] == 'PENDING - RETRO REVIEW') 
  ]

In [49]:
choices = [1]
cpts_new['is_retro'] = np.select(retro_conditions_new, choices, default=0)

In [50]:
## remove retros from list and drop 'is_retro' as it is no longer needed
cpts_new = cpts_new[cpts_new['is_retro']==0]

cpts_new.drop(columns='is_retro', inplace=True)

In [51]:
cpts_new['is_autoapp'] = np.where(cpts_new['status_name']=='APPROVED - AUTO', 1, 0)

In [52]:
cpts_new.PPL.fillna("N", inplace=True)

In [53]:
cpts_new['is_PPL'] = np.where(cpts_new['PPL']=='Y', 1, 0)

In [54]:
cpts_new['is_app'] = np.where(cpts_new['status_cat']=='APPROVED', 1, 0)

In [55]:
cpts_new['UNITS'] = 1

In [56]:
den_conditions_new = [
 (cpts_new['status_name'] == 'DENIED - CM') |
 (cpts_new['status_name'] == 'DENIED - BENEFIT CARVE OUT') |
 (cpts_new['status_name'] == 'DENIED - NOT A COVERED BENEFIT') |
 (cpts_new['status_name'] == 'DENIED - APPEAL') |
 (cpts_new['status_name'] == 'DENIED - CLINICAL TRIAL/EXP/INV') |
 (cpts_new['status_name'] == 'DENIED - TRANSPLANT') |
 (cpts_new['status_name'] == 'DENIED - MD') |
 (cpts_new['status_name'] == 'DENIED - CM/MD') |
 (cpts_new['status_name'] == 'DENIED - REDIRECT OSVN') |
 (cpts_new['status_name'] == 'DENIED - TICKLER')
  ]

In [57]:
choices = [1]
cpts_new['is_den'] = np.select(den_conditions_new, choices, default=0)

In [58]:
cpts_new = cpts_new[cpts_new['HCP_CONNECT_AUTH_NUMBER'].isna()==False].reset_index()

In [59]:
cpts_new.pivot_table(values='HCP_CONNECT_AUTH_NUMBER', index=['is_autoapp', 'is_den'], aggfunc='count', margins=True)

HCP_CONNECT_AUTH_NUMBER
is_autoapp is_den                         
0          0                        639658
           1                         16975
1          0                        294252
All                                 950885

In [60]:
cpts_new.drop(labels=['index', 'REF_TYPE_KEY', 'PPL', 'Date_Received', 'Date_Decision', 'status_cat'], axis=1, inplace=True)

In [61]:
list_o_specs = list(set(cpts['Specialty'].unique().tolist() + cpts_new['Specialty'].unique().tolist()))

## Switch Point - old referrals vs new referrals

In [62]:
#cpts_new = cpts

## Create cpts_manual

In [63]:
## This is a cpt_code level list of all manually reviewed referrals

In [64]:
cpts_manual = cpts[cpts['is_autoapp']==0]

In [65]:
cpts_manual.pivot_table(values='HCP_CONNECT_AUTH_NUMBER', index=['is_autoapp', 'is_den'], aggfunc='count', margins=True)

HCP_CONNECT_AUTH_NUMBER
is_autoapp is_den                         
0          0                       2532456
           1                         59258
All                                2591714

In [66]:
## Calculate Cost per manually reviewed CPT code

ga_cpt = 6500000 / cpts[cpts['is_autoapp']==0].shape[0]

In [67]:
ga_cpt = 2.53

In [68]:
## This is a grouped list of refs_manual that summarizes the volume of manually reviewed cpt_codes

In [69]:
cpts_manual = cpts_manual.groupby(['Specialty', 'CPT_Code', 'is_PPL'], as_index=False).agg({
    'UNITS' : 'count'
})

In [70]:
cpts_manual.rename(index=str, columns={'UNITS': 'UNITS_man'}, inplace=True)

In [71]:
cpts_manual['cost_to_review'] = cpts_manual['UNITS_man']*ga_cpt

## Create cptssum

In [72]:
cptssum = cpts.groupby(['Specialty', 'CPT_Code', 'is_PPL'], as_index=False).agg({
    'UNITS': 'count',
    'is_autoapp': 'mean',
    'is_den': 'mean',
})

In [73]:
cpts_w_claims0 = pd.merge(cptssum, cpts_manual, on=['Specialty', 'CPT_Code', 'is_PPL'], how='left')

In [74]:
cpts_w_claims0.head()

,Specialty,CPT_Code,is_PPL,UNITS,is_autoapp,is_den,UNITS_man,cost_to_review
0,ACUPUNCTURE,20550,1,1,0.0,0.000000,1.0,2.53
1,ACUPUNCTURE,20552,1,2,0.0,0.000000,2.0,5.06
2,ACUPUNCTURE,20610,1,2,0.0,0.000000,2.0,5.06
3,ACUPUNCTURE,2101,0,12,0.0,1.000000,12.0,30.36
4,ACUPUNCTURE,2101,1,7,0.0,0.857143,7.0,17.71


In [75]:
claims.head()

,Specialty,CPT_Code,cnt_hcp_cost,avg_hcp_cost,sd_hcp_cost
0,ACUPUNCTURE,97010,9,65.0000,0.000000
1,ACUPUNCTURE,97016,171,19.5935,16.730112
2,ACUPUNCTURE,97024,1,10.0000,NaN
3,ACUPUNCTURE,97026,252,4.6179,1.371409
4,ACUPUNCTURE,97110,7,41.4285,7.480132


In [76]:
cpts_w_claims1 = pd.merge(cpts_w_claims0, claims, on=['Specialty', 'CPT_Code'], how='left')

In [77]:
## For spec/cpt combos that don't have claims data associated, use the average of that cpt across specialties
cpts_w_claims2 = pd.merge(cpts_w_claims1, claims_sum, on='CPT_Code', how='left')

In [78]:
cpts_w_claims2['avg_hcp_cost_x'] = np.where(cpts_w_claims2['avg_hcp_cost_x'].isnull(), 
                                             cpts_w_claims2['avg_hcp_cost_y'],
                                             cpts_w_claims2['avg_hcp_cost_x'])

In [79]:
cpts_w_claims2.drop(columns=['avg_hcp_cost_y', 'sd_hcp_cost'], inplace=True)

In [80]:
cpts_w_claims2.rename(index=str, columns={'avg_hcp_cost_x': 'avg_hcp_cost'}, inplace=True)

In [81]:
cpts_w_claims2[cpts_w_claims2['Specialty']=='CARDIOLOGY'].head()

,Specialty,CPT_Code,is_PPL,UNITS,is_autoapp,is_den,UNITS_man,cost_to_review,cnt_hcp_cost,avg_hcp_cost
3748,CARDIOLOGY,0095T,0,1,0.000000,0.000000,1.0,2.53,NaN,NaN
3749,CARDIOLOGY,01922,0,2,0.000000,0.000000,2.0,5.06,NaN,604.377183
3750,CARDIOLOGY,0295T,0,51,0.000000,0.000000,51.0,129.03,4.0,73.437500
3751,CARDIOLOGY,0295T,1,132,0.833333,0.007576,22.0,55.66,4.0,73.437500
3752,CARDIOLOGY,0296T,0,100,0.000000,0.000000,100.0,253.00,38.0,63.700700


In [82]:
cpts_w_claims_fin = cpts_w_claims2

In [83]:
cpts_w_claims_fin[(cpts_w_claims_fin['UNITS_man'].isnull())].shape[0]

1726

## Calculate ROI

In [84]:
cpts_w_claims_fin['sum_cost_denied'] = cpts_w_claims_fin['is_den']*cpts_w_claims_fin['UNITS']*cpts_w_claims_fin['avg_hcp_cost']

In [85]:
cpts_w_claims_fin['UNITS_man'] = np.where(cpts_w_claims_fin['UNITS_man'].isnull(), 0, cpts_w_claims_fin['UNITS_man'])

In [86]:
cpts_w_claims_fin['ROI'] = cpts_w_claims_fin['sum_cost_denied']/cpts_w_claims_fin['cost_to_review']

In [87]:
cpts_w_claims_fin[(cpts_w_claims_fin['ROI'].isnull()) &
                 (cpts_w_claims_fin['Specialty']=='CARDIOLOGY')].head(15)

,Specialty,CPT_Code,is_PPL,UNITS,is_autoapp,is_den,UNITS_man,cost_to_review,cnt_hcp_cost,avg_hcp_cost,sum_cost_denied,ROI
3748,CARDIOLOGY,0095T,0,1,0.0,0.0,1.0,2.53,NaN,NaN,NaN,NaN
3760,CARDIOLOGY,0389T,1,1,1.0,0.0,0.0,NaN,2.0,51.31500,0.0,NaN
3763,CARDIOLOGY,0399T,1,4,1.0,0.0,0.0,NaN,NaN,456.35500,0.0,NaN
3764,CARDIOLOGY,0545F,1,1,1.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN
3765,CARDIOLOGY,1111F,1,1,1.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN
3768,CARDIOLOGY,300,0,0,0.0,0.0,0.0,0.00,NaN,308.81595,0.0,NaN
3769,CARDIOLOGY,3120F,1,2,0.0,0.0,2.0,5.06,NaN,NaN,NaN,NaN
3780,CARDIOLOGY,33220,0,1,0.0,0.0,1.0,2.53,NaN,NaN,NaN,NaN
3804,CARDIOLOGY,33271,0,1,0.0,0.0,1.0,2.53,NaN,NaN,NaN,NaN
3806,CARDIOLOGY,33273,0,1,0.0,0.0,1.0,2.53,NaN,NaN,NaN,NaN


In [88]:
cpts_w_claims_fin[(cpts_w_claims_fin['ROI'].isnull())].shape[0]

9815

In [89]:
## For groups where we don't know the average cost from 2018, the denominator of ROI is 0, and ROI is undefined. 
## Update the ROI for those to = 100 so they are NOT included in the dictionaries to auto-approve going forward.
cpts_w_claims_fin['ROI'] = np.where(cpts_w_claims_fin['avg_hcp_cost'].isnull(), 100, cpts_w_claims_fin['ROI'])

In [90]:
## For groups that were auto-approved at 100%, the denominator of ROI is 0, and ROI is undefined. 
## Update the ROI for those to = 0 so they are included in the dictionaries to auto-approve going forward.
cpts_w_claims_fin['ROI'] = np.where(cpts_w_claims_fin['ROI'].isnull(), 0.01, cpts_w_claims_fin['ROI'])
        

In [91]:
cpts_w_claims_fin['cost_to_review'].sum()

6323277.069999998

In [92]:
cpts_w_claims_fin['fin_aa_rec'] = np.where(cpts_w_claims_fin['ROI']<1, 1, 0)

## Create Dictionaries

In [93]:
## Model Additional Auto Approvals when different thresholds are set.
## Approach: use "given" threshold to determine which CPT codes are "auto-approve"-able for each specialty
##  - For loop through referrals, return 1 if all CPT codes are on "auto-approve"-able list, else 0 

In [94]:
list_o_types_to_pend = ['INPT ADM', 'EMERGENCY ROOM', 'DAY SURG', 'OOA INPT', 
                        'SKILLED NURSING', 'OBSERVATION', 'OB OBSERVATION']

In [95]:
## Function that gathers CPT codes that are auto-approvable and stores them in a dictionary.
def create_dict_of_CPT_codes(specialty_cpt, list_o_specs, PPL=1):
    spec_dict = {k: [] for k in list_o_specs}
    for index, row in specialty_cpt.iterrows():
        if row['fin_aa_rec'] == 1:
            if row['is_PPL'] == PPL:
                spec_dict[row['Specialty']].append(row['CPT_Code'])
    return spec_dict
            

In [96]:
## Function that gathers CPT codes that should be pended or added and stores them in a dictionary.
def create_dict_of_CPT_codes_v2(specialty_cpt, list_o_specs, PPL=1, approve=0):
    spec_dict = {k: [] for k in list_o_specs}
    for index, row in specialty_cpt.iterrows():
        if row['Decision'] == approve:
            if row['is_PPL'] == PPL:
                spec_dict[row['Specialty']].append(row['CPT_Code'])
    return spec_dict
            

In [97]:
cpts_w_claims_fin.reset_index(drop=True, inplace=True)

In [98]:
def collect_clinical_decisions(specialty_cpt, list_to_change, spec_list, rsn, new_status=0):
    decision_sources = cpts_w_claims_fin['dec_source'].to_list()
    overrule_reasons = cpts_w_claims_fin['overrule_rsn'].to_list()
    final_decisions = cpts_w_claims_fin['final_decision'].to_list()
    for index, row in specialty_cpt.iterrows():
        if row['Specialty'] in spec_list:
            if row['CPT_Code'] in list_to_change:
                decision_sources[index] = 'clinical'
                overrule_reasons[index] = rsn
                final_decisions[index] = new_status
    specialty_cpt['dec_source'] = decision_sources
    specialty_cpt['overrule_rsn'] = overrule_reasons
    specialty_cpt['final_decision'] = final_decisions
    return specialty_cpt
    

In [99]:
## Create new column in cpt_w_claims_fin that will store decision source.
cpts_w_claims_fin['dec_source'] = 'DBA'

In [100]:
## Create new column in cpt_w_claims_fin that will store clinical overrule_rsn
cpts_w_claims_fin['overrule_rsn'] = np.nan

In [101]:
## Create new column in cpt_w_claims_fin that will store final decision
cpts_w_claims_fin['final_decision'] = cpts_w_claims_fin['fin_aa_rec']

In [102]:
spec_dict_PPL = create_dict_of_CPT_codes(cpts_w_claims_fin, list_o_specs)

In [103]:
spec_dict_EPL = {k: [] for k in list_o_specs}

In [104]:
spec_dict_EPL = create_dict_of_CPT_codes(cpts_w_claims_fin, list_o_specs, PPL=0)

In [105]:
## Remove from EPL lists codes that don't appear in PPL list. 
## Spirit of this is to keep EPL lists more conservative
for spec in list_o_specs:
    epl_list = spec_dict_EPL[spec]
    ppl_list = spec_dict_PPL[spec]
    new_epl = [x for x in epl_list if x in ppl_list]
    spec_dict_EPL[spec] = new_epl

In [106]:
cpts_w_claims_fin[(cpts_w_claims_fin['Specialty']=='DERMATOLOGY') &
                  (cpts_w_claims_fin['is_PPL'] ==1) &
                 (cpts_w_claims_fin['final_decision']==1)].shape[0]

198

In [107]:
len(spec_dict_PPL['DERMATOLOGY'])

198

## Define codes to ADD or REMOVE from each list

The general process for creating dictionaries is to do the g&a analysis first.  These are noted with "DBA" (Decision by Analysis.

Next, in chronological order, we apply clinical decisions. In the first review session (Q2 2019) this was the order:
1. A list of PRL codes that Chris McRae said should be auto-approved
2. A list of PRL codes that Chris McRae said should be pended - "codestoremove"
3. Pend Specialties that should be pended categorically
4. Laurie and Chris's first round of mostly pends.  Rhoda also took a pass on 5/3/2019
5. Phyllis and Rhoda pass at pended specialties. 5/17/2019
6. Remove low volume codes.
7. Pend all EPL Codes

In [108]:
## ADD Derm codes - provided my Christel McRae (124)
derm_add0 = ['10060','10061','11000','11055','11057','11100','11101','11300','11301','11302','11303','11305',
            '11306','11307','11308','11310','11311','11312','11313','11404','11426','11440','11600',
            '11601','11602','11603','11604','11606','11620','11621','11622','11623','11624','11626','11640',
            '11641','11642','11643','11644','11646','11900','11901','29580','54050','54056','54100','67810',
            '69100','87101','87220','J3301','10061','10080','10081','10140','10180','11055','11056','11057',
            '11101','11300','11301','11302','11303','11305','11306','11307','11308','11310','11311','11312',
            '11313','11404','11426','11440','11444','11446','11600','11601','11602','11603','11604',
            '11606','11620','11621','11622','11623','11624','11626','11640','11641','11642','11643','11644',
            '11646','11900','11901','54105']


In [109]:
## find unique values (72)
derm_add = list(set(derm_add0))

In [110]:
## ADD Pain Add for PPL
pain_add0 = ['99203','99204','99213','99214']

In [111]:
## ADD Gastro and General Surgery for PPL and EPL
gi_add0 = ['45378','45380','45385','G0105','G0121','00812']

In [112]:
## ADD Blood Transfusions - across all specialties PPL and EPL
bt_add0 = ['36430','86900','86901','86902','86903','86904','86905','86906','86907','86908','86909','86910'
          ,'86911','86912','86913','86914','86915','86916','86917','86918','86919','86920']

In [113]:
## ADD Podiatry
podiatry_add0 = ['11055']

In [114]:
## REMOVE from all dictionaries
codes_to_remove0 = ['97810','95115','95116','95117','95170','95180','95181','95182','95183','95184','95185','95186',
                   '95187','95188','95199','95004','A0999','A0426','A0427','A0428','A0429','92590','92591','92592',
                   '92593','92594','90901','90902','90903','90904','90905','90906','90907','90908','90909','90910',
                   '90911','90875','90876','86890','93798','98940','J1050','58301','58565','58611','G0337',
                   '74740','89320','J3490','J0717','J1438','J7321','J7322','J7323','J7324','J7325','J3489','J0135',
                   'J3030','J1830','J9215','J9218','J0129','J9202','J3301','J1745','96365','96366','90378','J2505',
                   'J1440','J1441','J0885','J0886','T1013','96118','96119','96120','97166','97167','97168','97110',
                   '92015','97162','97163','97110','36468','92506','55250','59840',
                   'S0199','S0618','92595','58300','J7300','57170','A4266','11981',
                   'J7302','J7300','58670','59600','58605','64612','J0585','11055',
                   '11057','11719','11721','G0127','S0390']

In [115]:
codes_to_remove = list(set(codes_to_remove0))

In [116]:
cpts_w_claims_fin[(cpts_w_claims_fin['Specialty']=='GASTROENTEROLOGY') &
                  (cpts_w_claims_fin['is_PPL'] ==1) &
                 (cpts_w_claims_fin['final_decision']==1)].shape[0]

199

In [117]:
len(spec_dict_PPL['GASTROENTEROLOGY'])

199

In [118]:
cpts_w_claims_fin[(cpts_w_claims_fin['Specialty']=='DERMATOLOGY') &
                  (cpts_w_claims_fin['is_PPL'] ==1) &
                 (cpts_w_claims_fin['final_decision']==1)].shape[0]

198

In [119]:
len(spec_dict_PPL['DERMATOLOGY'])

198

In [120]:
cpts_w_claims_fin.pivot_table(values='CPT_Code', index=['final_decision'], aggfunc='count', margins=True)

,CPT_Code
final_decision,
0,7764
1,37221
All,44985


In [121]:
## Define function that takes specialty:cpt code list DICTIONARY, list of cpt
## codes, and "add" or "remove" parameter and adjusts the dictionary
def add_remove_cpt_codes_v2(spec_cpt_dict, spec_list, cpt_to_add_remove_dict, add=True):
    if add == True:
        for spec in spec_list:
            for code in cpt_to_add_remove_dict[spec]:
                if code in spec_cpt_dict[spec]:
                    continue
                else:
                    spec_cpt_dict[spec].append(code)
    else:
        for spec in spec_list:
            for code in cpt_to_add_remove_dict[spec]:
                if code in spec_cpt_dict[spec]:
                    spec_cpt_dict[spec].remove(code)
                else:
                    continue
    return spec_cpt_dict

In [122]:
## Define function that takes specialty:cpt code list, list of cpt
## codes, and "add" or "remove" parameter and adjusts the dictionary
def add_remove_cpt_codes_v1(spec_cpt_dict, specialty_cpt, spec_list, cpt_to_add_remove, add=True):
    if add == True:
        for spec in spec_list:
            for code in cpt_to_add_remove:
                if code in spec_cpt_dict[spec]:
                    continue
                else:
                    if code in specialty_cpt[specialty_cpt['Specialty']==spec]['CPT_Code'].to_list():
                        spec_cpt_dict[spec].append(code)
    else:
        for spec in spec_list:
            for code in cpt_to_add_remove:
                if code in spec_cpt_dict[spec]:
                    spec_cpt_dict[spec].remove(code)
                else:
                    continue
    return spec_cpt_dict 

#### Add specific codes to specific specialties

In [123]:
## List of codes to auto-approve add derm
spec_dict_PPL = add_remove_cpt_codes_v1(spec_dict_PPL, cpts_w_claims_fin, ['DERMATOLOGY'], derm_add)
spec_dict_EPL = add_remove_cpt_codes_v1(spec_dict_EPL, cpts_w_claims_fin, ['DERMATOLOGY'], derm_add)

In [124]:
derm_add

['11056',
 '11302',
 '10180',
 '11606',
 '10080',
 '11311',
 'J3301',
 '11603',
 '10060',
 '11306',
 '11642',
 '11643',
 '11646',
 '11620',
 '54100',
 '87101',
 '10081',
 '11307',
 '11601',
 '11444',
 '11623',
 '11900',
 '11901',
 '11101',
 '11057',
 '67810',
 '10061',
 '11404',
 '11055',
 '54056',
 '11308',
 '11644',
 '11301',
 '11100',
 '11426',
 '11313',
 '11640',
 '10140',
 '11602',
 '87220',
 '54050',
 '11300',
 '11641',
 '11305',
 '11000',
 '11310',
 '11604',
 '11626',
 '11621',
 '11440',
 '54105',
 '11446',
 '11600',
 '11312',
 '69100',
 '11622',
 '29580',
 '11303',
 '11624']

In [125]:
cpts_w_claims_fin.iloc[5048:5055]

,Specialty,CPT_Code,is_PPL,UNITS,is_autoapp,is_den,UNITS_man,cost_to_review,cnt_hcp_cost,avg_hcp_cost,sum_cost_denied,ROI,fin_aa_rec,dec_source,overrule_rsn,final_decision
5048,DERMATOLOGY,10040,0,443,0.000000,0.008989,443.0,1120.79,688.0,119.3472,475.243232,0.424025,1,DBA,NaN,1
5049,DERMATOLOGY,10040,1,446,0.930493,0.000000,31.0,78.43,688.0,119.3472,0.000000,0.000000,1,DBA,NaN,1
5050,DERMATOLOGY,10060,0,16,0.000000,0.000000,16.0,40.48,63.0,115.8563,0.000000,0.000000,1,DBA,NaN,1
5051,DERMATOLOGY,10060,1,58,0.948276,0.000000,3.0,7.59,63.0,115.8563,0.000000,0.000000,1,DBA,NaN,1
5052,DERMATOLOGY,10061,0,7,0.000000,0.000000,7.0,17.71,25.0,218.2792,0.000000,0.000000,1,DBA,NaN,1
5053,DERMATOLOGY,10061,1,25,1.000000,0.000000,0.0,NaN,25.0,218.2792,0.000000,0.010000,1,DBA,NaN,1
5054,DERMATOLOGY,10120,0,1,0.000000,0.000000,1.0,2.53,2.0,141.9200,0.000000,0.000000,1,DBA,NaN,1


In [126]:
## List of codes to auto-approve add derm
cpts_w_claims_fin = collect_clinical_decisions(cpts_w_claims_fin, derm_add, ['DERMATOLOGY'], rsn='from CM', new_status=1)

In [127]:
## List of codes to auto-approve add pain
spec_dict_PPL = add_remove_cpt_codes_v1(spec_dict_PPL, cpts_w_claims_fin, ['PAIN MANAGEMENT'], pain_add0)
spec_dict_EPL = add_remove_cpt_codes_v1(spec_dict_EPL, cpts_w_claims_fin, ['PAIN MANAGEMENT'], pain_add0)

In [128]:
## List of codes to auto-approve add pain
cpts_w_claims_fin = collect_clinical_decisions(cpts_w_claims_fin, pain_add0, ['PAIN MANAGEMENT'], rsn='from CM', new_status=1)

In [129]:
## List of codes to auto-approve add gastro
spec_dict_PPL = add_remove_cpt_codes_v1(spec_dict_PPL, cpts_w_claims_fin, ['GASTROENTEROLOGY','SURGERY - GENERAL'], gi_add0)
spec_dict_EPL = add_remove_cpt_codes_v1(spec_dict_EPL, cpts_w_claims_fin, ['GASTROENTEROLOGY','SURGERY - GENERAL'], gi_add0)

In [130]:
## List of codes to auto-approve add gastro
cpts_w_claims_fin = collect_clinical_decisions(cpts_w_claims_fin, gi_add0, 
                                               ['GASTROENTEROLOGY','SURGERY - GENERAL'], rsn='from CM', new_status=1)

In [131]:
## List of codes to auto-approve add blood transfusions
spec_dict_PPL = add_remove_cpt_codes_v1(spec_dict_PPL, cpts_w_claims_fin, list_o_specs, bt_add0)
spec_dict_EPL = add_remove_cpt_codes_v1(spec_dict_EPL, cpts_w_claims_fin, list_o_specs, bt_add0)

In [132]:
## List of codes to auto-approve add blood transfusions
cpts_w_claims_fin = collect_clinical_decisions(cpts_w_claims_fin, bt_add0, list_o_specs, rsn='from CM', new_status=1)

In [133]:
## List of codes to auto-approve add podiatry
spec_dict_PPL = add_remove_cpt_codes_v1(spec_dict_PPL, cpts_w_claims_fin, ['PODIATRY'], podiatry_add0)
spec_dict_EPL = add_remove_cpt_codes_v1(spec_dict_EPL, cpts_w_claims_fin, ['PODIATRY'], podiatry_add0)

In [134]:
## List of codes to auto-approve add podiatry
cpts_w_claims_fin = collect_clinical_decisions(cpts_w_claims_fin, podiatry_add0, ['PODIATRY'], rsn='from CM', new_status=1)

#### Pend specific set of codes across specialties

In [135]:
## List of codes to pend across specialties (PPL)
spec_dict_PPL = add_remove_cpt_codes_v1(spec_dict_PPL, cpts_w_claims_fin, list_o_specs, codes_to_remove, add=False)

In [136]:
## List of codes to pend across specialties (EPL)
spec_dict_EPL = add_remove_cpt_codes_v1(spec_dict_EPL, cpts_w_claims_fin, list_o_specs, codes_to_remove, add=False)

In [137]:
## List of codes to pend across specialties
cpts_w_claims_fin = collect_clinical_decisions(cpts_w_claims_fin, codes_to_remove, list_o_specs, rsn='from CM', new_status=0)

#### Pend Specialties categorically

In [138]:
## For specialties that UM clinical team deem necessary, empty dictionary
## To ensure that no referrals are auto-approved from it
specs_that_should_pend = ['ACUPUNCTURE', 'ADDICTION MEDICINE', 'ANESTHESIOLOGY',
                         'BEHAVIORAL HEALTH', 'CHIROPRACTIC', 'DENTIST', 'DME MAINTENANCE',
                         'LICENSED CLIN SOCIAL WORKER', 'MFCC (THERAPIST)', 'NON-CONTRACT UNKWN BILL AREA',
                         'NURSE PRACTITIONER', 'NURSING FACILITY - OTHER', 'OPTICIAN',
                        'PEDS-DEVELOPMENTAL BEHAVIORAL', 'PSYCHIATRY', 'PSYCHOLOGY', 'SENIOR WELLNESS VISIT',
                         'SPORTS MEDICINE', 'SURGERY - ORAL', 'AMBULATORY SURGICAL CENTER', 'AMBULANCE', 'ALLERGY/IMMUNOLOGY',
                         'CUSTODIAL CARE', 'HOME HEALTH', 'INFERTILITY', 'OPTOMETRY', 'PALLIATIVE CARE', 'PODIATRY', 'SLEEP STUDY',
                         'SNF - FAC', 'OCCUPATIONAL THERAPY', 'PHYSICAL THERAPY/REHAB', 'FACILITY SERVICES',
                         'GENETICS', 'INTERVENTIONAL RADIOLOGY', 'LABORATROY', 'NUCLEAR MEDICINE', 'ONCOLOGY - GYN',
                         'PATHOLOGY', 'PHARMACY', 'SURGERY - HAND']

In [139]:
for spec in specs_that_should_pend:
    cpts_w_claims_fin['overrule_rsn'] = np.where(cpts_w_claims_fin['Specialty'] == spec, 
                                    'Pend Specialty', cpts_w_claims_fin['overrule_rsn'])
    cpts_w_claims_fin['dec_source'] = np.where(cpts_w_claims_fin['Specialty']==spec, 
                                    'clinical', cpts_w_claims_fin['dec_source'])
    cpts_w_claims_fin['final_decision'] = np.where(cpts_w_claims_fin['Specialty']==spec, 
                                    0, cpts_w_claims_fin['final_decision'])

In [140]:
def pend_specs_by_spec(list_o_specs, spec_dict):
    for spec in list_o_specs:
        spec_dict[spec] = []
    return spec_dict

In [141]:
spec_dict_PPL = pend_specs_by_spec(specs_that_should_pend, spec_dict_PPL)

In [142]:
spec_dict_EPL = pend_specs_by_spec(specs_that_should_pend, spec_dict_EPL)

In [143]:
cpts_w_claims_fin.head()

,Specialty,CPT_Code,is_PPL,UNITS,is_autoapp,is_den,UNITS_man,cost_to_review,cnt_hcp_cost,avg_hcp_cost,sum_cost_denied,ROI,fin_aa_rec,dec_source,overrule_rsn,final_decision
0,ACUPUNCTURE,20550,1,1,0.0,0.000000,1.0,2.53,NaN,53.542567,0.0,0.0,1,clinical,Pend Specialty,0
1,ACUPUNCTURE,20552,1,2,0.0,0.000000,2.0,5.06,NaN,63.492905,0.0,0.0,1,clinical,Pend Specialty,0
2,ACUPUNCTURE,20610,1,2,0.0,0.000000,2.0,5.06,NaN,144.174081,0.0,0.0,1,clinical,Pend Specialty,0
3,ACUPUNCTURE,2101,0,12,0.0,1.000000,12.0,30.36,NaN,NaN,NaN,100.0,0,clinical,Pend Specialty,0
4,ACUPUNCTURE,2101,1,7,0.0,0.857143,7.0,17.71,NaN,NaN,NaN,100.0,0,clinical,Pend Specialty,0


In [144]:
clinical_decisions_20190503_remove.head()

,Specialty,CPT_Code,Decision,Notes,is_PPL,overturned
0,OPHTHALMOLOGY,J7312,0,overutilization concern,1,1
1,OPHTHALMOLOGY,J9035,0,overutilization concern,1,1
2,PHARMACY,99601,0,overutilization concern,1,1
3,RHEUMATOLOGY,77080,0,overutilization concern,1,1
4,SURGERY - COLONRECTAL,45300,0,overutilization concern,1,1


#### Remove codes chosen by Laurie and Christel 5/03/2019

In [145]:
clinical_decisons_20190503_remove = create_dict_of_CPT_codes_v2(clinical_decisions_20190503_remove, list_o_specs)

In [146]:
list_o_specs_20190503_remove = clinical_decisions_20190503_remove['Specialty'].unique().tolist()

In [147]:
cpts_w_claims_fin = cpts_w_claims_fin.merge(clinical_decisions_20190503_remove, how='left', on=['Specialty', 'CPT_Code', 'is_PPL'])

In [148]:
cpts_w_claims_fin['Decision'] = np.where(cpts_w_claims_fin['Decision'].isna(), cpts_w_claims_fin['final_decision'], cpts_w_claims_fin['Decision'])

In [149]:
cpts_w_claims_fin[cpts_w_claims_fin['Decision'].isna()==True].shape[0]

0

In [150]:
cpts_w_claims_fin[cpts_w_claims_fin['Decision'].isna()==False].shape[0]

44987

In [151]:
cpts_w_claims_fin['Decision'].unique()

array([0., 1.])

In [152]:
cpts_w_claims_fin.pivot_table('CPT_Code', index='Decision', columns='final_decision', aggfunc='count')

final_decision,0,1
Decision,,
0.0,23399,488
1.0,134,20966


In [153]:
cpts_w_claims_fin[(cpts_w_claims_fin['final_decision']!=cpts_w_claims_fin['Decision'])].head()

,Specialty,CPT_Code,is_PPL,UNITS,is_autoapp,is_den,UNITS_man,cost_to_review,cnt_hcp_cost,avg_hcp_cost,sum_cost_denied,ROI,fin_aa_rec,dec_source,overrule_rsn,final_decision,Decision,Notes,overturned
3755,CARDIOLOGY,0297T,1,58,0.728814,0.000000,15.0,37.95,19.0,43.365700,0.00000,0.000000,1,DBA,NaN,1,0.0,needs review,1.0
3757,CARDIOLOGY,0298T,1,62,0.709677,0.016129,18.0,45.54,42.0,33.392300,33.39230,0.733252,1,DBA,NaN,1,0.0,needs review,1.0
3965,CARDIOLOGY,76770,1,33,1.000000,0.000000,0.0,NaN,22.0,68.795400,0.00000,0.010000,1,DBA,NaN,1,0.0,inappropriate loc,1.0
3991,CARDIOLOGY,78452,1,2577,0.859193,0.000388,362.0,915.86,1650.0,459.553000,459.37474,0.501577,1,DBA,NaN,1,0.0,overutilization concern,1.0
4009,CARDIOLOGY,80048,1,34,0.970588,0.000000,1.0,2.53,NaN,33.854175,0.00000,0.000000,1,DBA,NaN,1,0.0,inappropriate loc,1.0


In [154]:
cpts_w_claims_fin['overrule_rsn'].unique()

array(['Pend Specialty', nan, 'from CM'], dtype=object)

In [155]:
## Remove codes from dictionaries as chosen by Laurie and Christel
spec_dict_PPL = add_remove_cpt_codes_v2(spec_dict_PPL, list_o_specs_20190503_remove, clinical_decisons_20190503_remove, add=False)

In [156]:
## Pend codes decided by clinicians
cpts_w_claims_fin['overrule_rsn'] = np.where(cpts_w_claims_fin['Decision']!=cpts_w_claims_fin['final_decision'], 
                                    cpts_w_claims_fin['Notes'], cpts_w_claims_fin['overrule_rsn'])

In [157]:
cpts_w_claims_fin['dec_source'] = np.where(cpts_w_claims_fin['Decision']!=cpts_w_claims_fin['final_decision'], 
                                    'clinical', cpts_w_claims_fin['dec_source'])

In [158]:
cpts_w_claims_fin['final_decision'] = np.where(cpts_w_claims_fin['Decision']!=cpts_w_claims_fin['final_decision'], 
                                    cpts_w_claims_fin['Decision'], cpts_w_claims_fin['final_decision'])

In [159]:
cpts_w_claims_fin.drop(['Decision', 'Notes', 'overturned'], axis=1, inplace=True)

In [160]:
cpts_w_claims_fin[(cpts_w_claims_fin['Specialty']=='GASTROENTEROLOGY') &
                  (cpts_w_claims_fin['is_PPL'] ==1) &
                 (cpts_w_claims_fin['final_decision']==1)].shape[0]

187

In [161]:
len(spec_dict_PPL['GASTROENTEROLOGY'])

190

#### Add codes chosen by Laurie and Christel 5/03/2019

In [ ]:
clinical_decisons_20190503_add = create_dict_of_CPT_codes_v2(clinical_decisions_20190503_add, list_o_specs)

In [ ]:
list_o_specs_20190503_add = clinical_decisions_20190503_add['Specialty'].unique().tolist()

In [162]:
cpts_w_claims_fin = cpts_w_claims_fin.merge(clinical_decisions_20190503_add, how='left', on=['Specialty', 'CPT_Code', 'is_PPL'])

In [167]:
cpts_w_claims_fin[cpts_w_claims_fin['Decision']==1]

,Specialty,CPT_Code,is_PPL,UNITS,is_autoapp,is_den,UNITS_man,cost_to_review,cnt_hcp_cost,avg_hcp_cost,sum_cost_denied,ROI,fin_aa_rec,dec_source,overrule_rsn,final_decision,Decision,Notes,overturned
4056,CARDIOLOGY,82948,1,47,0.000000,0.000000,47.0,118.91,NaN,9.968744,0.000000,0.000000,1,DBA,NaN,1.0,1.0,NaN,0.0
4168,CARDIOLOGY,93000,1,19819,0.562614,0.000050,8668.0,21930.04,14785.0,19.227700,19.226730,0.000877,1,DBA,NaN,1.0,1.0,NaN,0.0
4170,CARDIOLOGY,93005,1,68,0.191176,0.000000,55.0,139.15,33.0,10.590300,0.000000,0.000000,1,DBA,NaN,1.0,1.0,NaN,0.0
4172,CARDIOLOGY,93010,1,130,0.276923,0.000000,94.0,237.82,23954.0,10.840900,0.000000,0.000000,1,DBA,NaN,1.0,1.0,NaN,0.0
4174,CARDIOLOGY,93015,1,5144,0.881244,0.000000,610.0,1543.30,2129.0,73.612700,0.000000,0.000000,1,DBA,NaN,1.0,1.0,NaN,0.0
4176,CARDIOLOGY,93016,1,159,0.187500,0.006250,129.0,326.37,1333.0,22.818400,22.675785,0.069479,1,DBA,NaN,1.0,1.0,NaN,0.0
4178,CARDIOLOGY,93017,1,102,0.303922,0.000000,71.0,179.63,11.0,48.871800,0.000000,0.000000,1,DBA,NaN,1.0,1.0,NaN,0.0
4180,CARDIOLOGY,93018,1,619,0.645161,0.003226,219.0,554.07,1525.0,15.820400,31.589766,0.057014,1,DBA,NaN,1.0,1.0,NaN,0.0
4183,CARDIOLOGY,93040,1,610,0.103279,0.000000,547.0,1383.91,133.0,11.329500,0.000000,0.000000,1,DBA,NaN,1.0,1.0,NaN,0.0
4189,CARDIOLOGY,93224,1,2652,0.906109,0.000000,249.0,629.97,1337.0,93.790500,0.000000,0.000000,1,DBA,NaN,1.0,1.0,NaN,0.0


In [169]:
cpts_w_claims_fin.pivot_table('CPT_Code', index='Decision', columns='final_decision', aggfunc='count')

final_decision,0.0,1.0
Decision,,
0.0,23887.0,NaN
1.0,NaN,21102.0


In [168]:
cpts_w_claims_fin['Decision'] = np.where(cpts_w_claims_fin['Decision'].isna(), cpts_w_claims_fin['final_decision'], cpts_w_claims_fin['Decision'])

In [ ]:
clinical_decisions_20190503_add.head()

In [ ]:
cpts_w_claims_fin.head()

In [ ]:
## Remove codes from dictionaries as chosen by Laurie and Christel
spec_dict_PPL = add_remove_cpt_codes_v2(spec_dict_PPL, list_o_specs_20190503_add, clinical_decisons_20190503_add, add=True)

In [170]:
## Pend codes decided by clinicians
cpts_w_claims_fin['overrule_rsn'] = np.where(cpts_w_claims_fin['Decision']!=cpts_w_claims_fin['final_decision'], 
                                    cpts_w_claims_fin['Notes'], cpts_w_claims_fin['overrule_rsn'])

In [171]:
cpts_w_claims_fin['dec_source'] = np.where(cpts_w_claims_fin['Decision']!=cpts_w_claims_fin['final_decision'], 
                                    'clinical', cpts_w_claims_fin['dec_source'])

In [172]:
cpts_w_claims_fin['final_decision'] = np.where(cpts_w_claims_fin['Decision']!=cpts_w_claims_fin['final_decision'], 
                                    cpts_w_claims_fin['Decision'], cpts_w_claims_fin['final_decision'])

In [173]:
cpts_w_claims_fin.drop(['Decision', 'Notes', 'overturned'], axis=1, inplace=True)

In [ ]:
cpts_w_claims_fin[(cpts_w_claims_fin['Specialty']=='GASTROENTEROLOGY') &
                  (cpts_w_claims_fin['is_PPL'] ==1) &
                 (cpts_w_claims_fin['final_decision']==1)].shape[0]

In [ ]:
len(spec_dict_PPL['GASTROENTEROLOGY'])

In [ ]:
cpts_w_claims_fin[cpts_w_claims_fin['overrule_rsn']=='upcode']

#### Add codes chosen by Phyllis and Rhoda 5/17/2019

In [ ]:
clinical_decisions_20190517_add = create_dict_of_CPT_codes_v2(clinical_decisions_20190517, list_o_specs, PPL=1, approve=1)

In [ ]:
list_o_specs_20190517_add = clinical_decisions_20190517['Specialty'].unique().tolist()

In [174]:
cpts_w_claims_fin = cpts_w_claims_fin.merge(clinical_decisions_20190517, how='left', on=['Specialty', 'CPT_Code', 'is_PPL'])

In [180]:
cpts_w_claims_fin.pivot_table('CPT_Code', index='Decision', columns='final_decision', aggfunc='count')

final_decision,0.0,1.0
Decision,,
0.0,23836.0,NaN
1.0,NaN,21153.0


In [176]:
cpts_w_claims_fin['Decision'] = np.where(cpts_w_claims_fin['Decision'].isna(), cpts_w_claims_fin['final_decision'], cpts_w_claims_fin['Decision'])

In [ ]:
## Remove codes from dictionaries as chosen by Laurie and Christel
spec_dict_PPL = add_remove_cpt_codes_v2(spec_dict_PPL, list_o_specs_20190517_add, clinical_decisions_20190517_add, add=True)

In [177]:
## Pend codes decided by clinicians
cpts_w_claims_fin['overrule_rsn'] = np.where(cpts_w_claims_fin['Decision']!=cpts_w_claims_fin['final_decision'], 
                                    cpts_w_claims_fin['Notes'], cpts_w_claims_fin['overrule_rsn'])

In [178]:
cpts_w_claims_fin['dec_source'] = np.where(cpts_w_claims_fin['Decision']!=cpts_w_claims_fin['final_decision'], 
                                    'clinical', cpts_w_claims_fin['dec_source'])

In [179]:
cpts_w_claims_fin['final_decision'] = np.where(cpts_w_claims_fin['Decision']!=cpts_w_claims_fin['final_decision'], 
                                    cpts_w_claims_fin['Decision'], cpts_w_claims_fin['final_decision'])

In [181]:
cpts_w_claims_fin.drop(['Decision', 'Notes', 'overturned'], axis=1, inplace=True)

In [182]:
cpts_w_claims_fin[
                 (cpts_w_claims_fin['overrule_rsn']=='PA and RL review of pend specs')].shape[0]

51

#### codes chosen by Laurie and Rhoda 5/22/2019

In [ ]:
clinical_decisions_20190522 = create_dict_of_CPT_codes_v2(clinical_decisions_20190517, list_o_specs, PPL=1, approve=1)

In [ ]:
list_o_specs_20190517_add = clinical_decisions_20190517['Specialty'].unique().tolist()

In [183]:
cpts_w_claims_fin = cpts_w_claims_fin.merge(clinical_decisions_20190522, how='left', on=['Specialty', 'CPT_Code', 'is_PPL'])

In [184]:
cpts_w_claims_fin.pivot_table('CPT_Code', index='Decision', columns='final_decision', aggfunc='count')

final_decision,0.0,1.0
Decision,,
0.0,1,17
1.0,14,25


In [185]:
cpts_w_claims_fin['Decision'] = np.where(cpts_w_claims_fin['Decision'].isna(), cpts_w_claims_fin['final_decision'], cpts_w_claims_fin['Decision'])

In [ ]:
## Remove codes from dictionaries as chosen by Laurie and Christel
spec_dict_PPL = add_remove_cpt_codes_v2(spec_dict_PPL, list_o_specs_20190517_add, clinical_decisions_20190517_add, add=True)

In [186]:
## Pend codes decided by clinicians
cpts_w_claims_fin['overrule_rsn'] = np.where(cpts_w_claims_fin['Decision']!=cpts_w_claims_fin['final_decision'], 
                                    cpts_w_claims_fin['Notes'], cpts_w_claims_fin['overrule_rsn'])

In [187]:
cpts_w_claims_fin['dec_source'] = np.where(cpts_w_claims_fin['Decision']!=cpts_w_claims_fin['final_decision'], 
                                    'clinical', cpts_w_claims_fin['dec_source'])

In [188]:
cpts_w_claims_fin['final_decision'] = np.where(cpts_w_claims_fin['Decision']!=cpts_w_claims_fin['final_decision'], 
                                    cpts_w_claims_fin['Decision'], cpts_w_claims_fin['final_decision'])

In [189]:
cpts_w_claims_fin.drop(['Decision', 'Notes'], axis=1, inplace=True)

In [190]:
cpts_w_claims_fin[
                 (cpts_w_claims_fin['overrule_rsn']=='PA and RL review of pend specs')].shape[0]

51

#### Pend codes with fewer than 30 in the past year

In [191]:
## Pend codes that have volume < 30
cpts_w_claims_fin['overrule_rsn'] = np.where(cpts_w_claims_fin['UNITS']<=30, 
                                    'low volume', cpts_w_claims_fin['overrule_rsn'])

In [192]:
cpts_w_claims_fin['dec_source'] = np.where(cpts_w_claims_fin['UNITS']<=30, 
                                    'Rule', cpts_w_claims_fin['dec_source'])

In [193]:
cpts_w_claims_fin['final_decision'] = np.where(cpts_w_claims_fin['UNITS']<=30, 
                                    0, cpts_w_claims_fin['final_decision'])

In [ ]:
def remove_low_vol_codes(specialty_cpt, spec_dict, list_o_specs, threshold=30, PPL=1):
    low_vol_dict = {k: [] for k in list_o_specs}
    low_vol_specs = []
    for index, row in specialty_cpt.iterrows():
        if row['is_PPL'] == PPL:
            if row['UNITS'] <= threshold:
                low_vol_dict[row['Specialty']].append(row['CPT_Code'])
                low_vol_specs.append(row['Specialty'])
    low_vol_specs = list(set(low_vol_specs))
    for spec in low_vol_specs:
        for code in low_vol_dict[spec]:
            if code in spec_dict[spec]:
                spec_dict[spec].remove(code)
            else:
                continue
    return spec_dict
            

In [ ]:
spec_dict_PPL = remove_low_vol_codes(cpts_w_claims_fin, spec_dict_PPL, list_o_specs)

In [194]:
cpts_w_claims_fin[(cpts_w_claims_fin['Specialty']=='GASTROENTEROLOGY') &
                  (cpts_w_claims_fin['is_PPL'] ==1) &
                 (cpts_w_claims_fin['final_decision']==1)].shape[0]

36

In [195]:
len(spec_dict_PPL['GASTROENTEROLOGY'])

190

#### Pend all EPL

In [196]:
## Pend EPL
cpts_w_claims_fin['overrule_rsn'] = np.where(cpts_w_claims_fin['is_PPL']==0, 
                                    'Pend EPL', cpts_w_claims_fin['overrule_rsn'])

In [197]:
cpts_w_claims_fin['dec_source'] = np.where(cpts_w_claims_fin['is_PPL']==0, 
                                    'clinical', cpts_w_claims_fin['dec_source'])

In [198]:
cpts_w_claims_fin['final_decision'] = np.where(cpts_w_claims_fin['is_PPL']==0, 
                                    0, cpts_w_claims_fin['final_decision'])

In [ ]:
for spec in list_o_specs:
    spec_dict_EPL[spec] = []

In [200]:
## Function that gathers CPT codes that should be pended and stores them in a dictionary.
def create_dict_of_CPT_codes_v3(specialty_cpt, list_o_specs, PPL=1):
    spec_dict = {k: [] for k in list_o_specs}
    for index, row in specialty_cpt.iterrows():
        if row['final_decision'] == 1:
            if row['is_PPL'] == PPL:
                spec_dict[row['Specialty']].append(row['CPT_Code'])
    return spec_dict

In [201]:
new_spec_dictionary_PPL = create_dict_of_CPT_codes_v3(cpts_w_claims_fin, list_o_specs, PPL=1)

In [202]:
len(new_spec_dictionary_PPL['GASTROENTEROLOGY'])

36

In [ ]:
#### Prepare a newer set of referrals to run through the new code.

In [203]:
## Assigns an auto-approve (1) or pend (0) status to each specialty / code group
def assign_status(codes, spec_dict_PPL, spec_dict_EPL, list_of_types_pend):
    status = list(np.zeros(codes.shape[0]))
    specs_list = codes['Specialty'].unique().tolist()
    print(len(status))
    print(codes.shape[0])
    for spec in specs_list:
        if spec not in spec_dict_PPL:
            spec_dict_PPL[spec] = []
    for spec in specs_list:
        if spec not in spec_dict_EPL:
            spec_dict_EPL[spec] = []
    for index, row in codes.iterrows():
        if row['is_PPL'] == 1:
            if row['CPT_Code'] in spec_dict_PPL[row['Specialty']]:
                if row['ref_type'] not in list_of_types_pend:
                    status[index] = 1
        ##else:
          ##  if row['CPT_Code'] in spec_dict_EPL[row['Specialty']]:
            ##    status[index] = 0
    return status

In [204]:
cpts_new.reset_index(drop=True, inplace=True)

In [205]:
auto_approve = assign_status(cpts_new, new_spec_dictionary_PPL, 
                             spec_dict_EPL, list_o_types_to_pend)

950885
950885


In [206]:
cpts_new['auto_approvable'] = auto_approve

In [207]:
## Find which (REFERRALS) not CPT codes have all cpt codes in the dictionary and thus, "auto-approvable"

In [208]:
refs_results = cpts_new.groupby(['HCP_CONNECT_AUTH_NUMBER'], as_index=False).agg({'auto_approvable': 'mean',
                                                                                 'is_den': 'mean'})

In [209]:
refs_results['aa-yn'] = np.where(refs_results['auto_approvable']==1, 1, 0)

In [210]:
refs_results.drop('auto_approvable', axis=1, inplace=True)

In [211]:
refs_results.pivot_table('HCP_CONNECT_AUTH_NUMBER', index='aa-yn', columns='is_den', aggfunc='count', margins=True)

is_den,0,1,All
aa-yn,,,
0,251885,10758,262643
1,287672,660,288332
All,539557,11418,550975


In [212]:
refs_results.drop('is_den', axis=1, inplace=True)

In [213]:
cpts_new = cpts_new.merge(refs_results, how='left', on=['HCP_CONNECT_AUTH_NUMBER'])

In [214]:
cpts_w_claims_fin[cpts_w_claims_fin['CPT_Code']=='J1561'].head()

,Specialty,CPT_Code,is_PPL,UNITS,is_autoapp,is_den,UNITS_man,cost_to_review,cnt_hcp_cost,avg_hcp_cost,sum_cost_denied,ROI,fin_aa_rec,dec_source,overrule_rsn,final_decision
16855,FACILITY SERVICES,J1561,0,12,0.0,0.250000,12.0,30.36,105.0,2019.270300,6057.810900,199.532638,0,clinical,Pend EPL,0.0
16856,FACILITY SERVICES,J1561,1,1,0.0,0.000000,1.0,2.53,105.0,2019.270300,0.000000,0.000000,1,Rule,low volume,0.0
19917,HEMATOLOGY/ONCOLOGY,J1561,0,13,0.0,0.153846,13.0,32.89,NaN,3831.021686,7662.043371,232.959665,0,clinical,Pend EPL,0.0
20809,HOME HEALTH,J1561,0,37,0.0,0.000000,37.0,93.61,20.0,4668.504000,0.000000,0.000000,1,clinical,Pend EPL,0.0
20810,HOME HEALTH,J1561,1,14,0.0,0.000000,14.0,35.42,20.0,4668.504000,0.000000,0.000000,1,Rule,low volume,0.0


In [215]:
cpts_new.pivot_table(values='UNITS', index=['is_autoapp', 'is_den'], columns='aa-yn', aggfunc='count', margins=True)##.to_csv('../Data/Outputs/costs_cm.csv')

aa-yn                   0       1     All
is_autoapp is_den                        
0          0       447316  192342  639658
           1        15933    1042   16975
1          0        33081  261171  294252
All                496330  454555  950885

In [216]:
# cpts_new[(cpts_new['is_autoapp']==1) &
#          (cpts_new['aa-yn']==0)].pivot_table(values='UNITS', index = 'is_den',
#                     aggfunc='count', margins=True).to_csv('../Data/Outputs/2019_spec_cpt_results.csv', sep='|')

## Construct Specialty Summary

In [217]:
refs_new_head = cpts_new.groupby(['Specialty', 'ref_type', 'region', 'HCP_CONNECT_AUTH_NUMBER'], as_index=False).agg({
    'UNITS': 'count',
    'is_autoapp': 'mean',
    'is_PPL': 'mean',
    'is_app': 'mean',
    'is_den': 'mean',
    'aa-yn': 'mean'
})

In [218]:
refs_new_head.pivot_table(values=['is_autoapp', 'aa-yn'], index='Specialty', aggfunc=['count', 'mean']).to_csv('../Data/Outputs/spec_rates.csv',
                                                                                                              sep='|')

In [219]:
specialty_summary = pd.read_csv('../Data/Outputs/spec_rates.csv', sep='|', skiprows=3, names=['Specialty',
                                                                                             'ref_vol', 'ref_vol2',
                                                                                             'new_rate', 'old_rate'])

In [220]:
specialty_summary.drop('ref_vol2', axis=1, inplace=True)

In [221]:
## Get counts of codes for each specialty

In [222]:
def spec_code_counts(spec_dict, spec_summary):
    counts = []
    for index, row in spec_summary.iterrows():
        counts.append(len(spec_dict[row['Specialty']]))
    return counts

In [223]:
PPL_counts = spec_code_counts(spec_dict_PPL, specialty_summary)

In [224]:
specialty_summary['num_codes']=PPL_counts

In [225]:
## get 'denied now approved'

In [226]:
refs_new_head['denied_now_aa'] = refs_new_head['is_den'] * refs_new_head['aa-yn']

In [227]:
refs_new_head.pivot_table('denied_now_aa', index='Specialty', aggfunc='sum').to_csv('../Data/Outputs/spec_den_now_aa.csv',
                                                                                                      sep='|')

In [228]:
ss_deny_now_aa = pd.read_csv('../Data/Outputs/spec_den_now_aa.csv', sep='|', skiprows=2, names=['Specialty', 'deny_now_aa'])

In [229]:
specialty_summary = specialty_summary.merge(ss_deny_now_aa, how='inner', on='Specialty')

In [230]:
## Prep cpts_w_claims_fin to merge into cpts_new

In [231]:
cpts_w_claims_fin_select = cpts_w_claims_fin.drop(['UNITS', 'is_autoapp', 'is_den',
       'UNITS_man', 'cost_to_review', 'cnt_hcp_cost',
       'sum_cost_denied'], axis=1)

In [232]:
cpts_new_w_fin = pd.merge(cpts_new, cpts_w_claims_fin_select, how='left', on=['Specialty', 'CPT_Code', 'is_PPL'])

In [233]:
cpts_new_w_fin['dollars_denied_now_app'] = cpts_new_w_fin['is_den']*cpts_new_w_fin['aa-yn']*cpts_new_w_fin['avg_hcp_cost']

In [234]:
cpts_new_w_fin['cost_to_review_bene'] = (1-cpts_new_w_fin['is_autoapp'])*cpts_new_w_fin['aa-yn']*ga_cpt

In [235]:
cpts_new_w_fin['cost_to_review_loss'] = cpts_new_w_fin['is_autoapp']*(1-cpts_new_w_fin['aa-yn'])*ga_cpt

In [236]:
cpts_new_w_fin['new_denial_bene_est'] = cpts_new_w_fin['is_autoapp']*(1-cpts_new_w_fin['aa-yn'])*ga_cpt*cpts_new_w_fin['ROI']*cpts_new_w_fin['is_PPL']

In [237]:
cpts_new_w_fin.pivot_table(values=['dollars_denied_now_app', 'cost_to_review_bene', 'cost_to_review_loss',
                                  'new_denial_bene_est'], index='Specialty', aggfunc='sum').to_csv('../Data/Outputs/specialty_fins.csv', sep='|')

In [238]:
specialty_fins = pd.read_csv('../Data/Outputs/specialty_fins.csv', sep='|')

In [239]:
specialty_summary = specialty_summary.merge(specialty_fins, how='inner', on='Specialty')

In [240]:
specialty_summary['net_benefit'] = specialty_summary['cost_to_review_bene'] - specialty_summary['cost_to_review_loss'] - specialty_summary['dollars_denied_now_app']

In [241]:
specialty_summary['net_benefit_w_new_den'] = specialty_summary['cost_to_review_bene'] - specialty_summary['cost_to_review_loss'] -specialty_summary['dollars_denied_now_app'] + specialty_summary['new_denial_bene_est']

In [242]:
# Give the filename you wish to save the file to
spec_summary_filename = '../Data/Outputs/specialty_summary_fin.xlsx'

# Use this function to search for any files which match your filename
files_present = os.path.isfile(spec_summary_filename)

# if no matching files, write to csv, if there are matching files, print statement
if not files_present:
    specialty_summary.to_excel(spec_summary_filename, index=False, float_format='%.2f',
                          header=['Specialty', 'volume', 'new rate', 'old_rate', '# codes in dictionary', 'false positives',
                                 'cost to review, benefit', 'cost to review, loss', 'dollars denied, now approved',
                                 'new denials, benefit (est)', 'new benefit', 'net benefit, est'], 
                          freeze_panes=(1,0))
else:
    print('WARNING: This file already exists!')

In [243]:
cpts_w_claims_fin['overrule_rsn'].unique()

array(['low volume', 'Pend EPL', 'Pend Specialty',
       'PA and RL review of pend specs', nan, 'from CM', 'needs review',
       'inappropriate loc', 'overutilization concern', 'guideline req',
       'facility code', 'Rhoda 5/23', 'bene check', 'heavy cap vol',
       'cost containment', 'unclassified drug', 'RNL', 'Laurie 5/22'],
      dtype=object)

In [244]:
refs_results['aa-yn'].mean()

0.5233123099959164

In [ ]:
## rate before laurie and chrisel's codes to pend
v0 = 1198038/2086468
print(v0)

In [ ]:
## rate after laurie and christels' codes to pend
v1 = 1091109/2086468
print(v1)

In [ ]:
v2 = 1097619/2086468
print(v2)

In [ ]:
## using 2018 data to create rules, and then applying them to early 2019 data
v3 = 274250/543258
print(v3)

In [ ]:
## using clinical review list from 20190501 from Rhoda 
.401

In [ ]:
## 20190503 V1 - More Conservative Version
.4532

In [ ]:
## 20190503 v2 - Baseline
.462

In [ ]:
## 20190517 - after Phyllis and Rhoda add back several codes
.52

## Code Detail List

In [245]:
spec_cpt_w_projected_aa_rate = cpts_new.groupby(['Specialty', 'CPT_Code', 'is_PPL'], as_index=False).agg({
    'aa-yn': 'mean'
})

In [246]:
cpts_w_claims_fin = cpts_w_claims_fin.merge(spec_cpt_w_projected_aa_rate, how='left', on=['Specialty', 'CPT_Code', 'is_PPL'])

In [247]:
cpts_new['UNITS'].sum()

950885

In [248]:
# Give the filename you wish to save the file to
spec_cpt_w_projected_aa_rate_fin_filename = '../Data/Outputs/spec_cpt_w_projected_aa_rate.xlsx'

# Use this function to search for any files which match your filename
files_present = os.path.isfile(spec_cpt_w_projected_aa_rate_fin_filename)

# if no matching files, write to csv, if there are matching files, print statement
if not files_present:
    spec_cpt_w_projected_aa_rate.to_excel(spec_cpt_w_projected_aa_rate_fin_filename, index=False, float_format='%.2f',
                          header=['Specialty', 'CPT Code', 'PPL?', 'new rate'], 
                          freeze_panes=(1,0))
else:
    print('WARNING: This file already exists!')

In [249]:
cpts_w_claims_fin[(cpts_w_claims_fin['Specialty']=='DERMATOLOGY') &
                 (cpts_w_claims_fin['CPT_Code']=='99203')]

,Specialty,CPT_Code,is_PPL,UNITS,is_autoapp,is_den,UNITS_man,cost_to_review,cnt_hcp_cost,avg_hcp_cost,sum_cost_denied,ROI,fin_aa_rec,dec_source,overrule_rsn,final_decision,aa-yn
5455,DERMATOLOGY,99203,0,5550,0.001612,0.012894,5541.0,14018.73,4772.0,94.7149,6777.950222,0.483492,1,clinical,Pend EPL,0.0,0.000000
5456,DERMATOLOGY,99203,1,47190,0.781776,0.005743,10298.0,26053.94,4772.0,94.7149,25667.737900,0.985177,1,DBA,NaN,1.0,0.992183


In [250]:
cpts_new.pivot_table('CPT_Code', index='aa-yn', columns='is_den', aggfunc='count', margins=True)

is_den,0,1,All
aa-yn,,,
0,480397,15933,496330
1,453513,1042,454555
All,933910,16975,950885


## Spot Check Specific Referrals

In [251]:
cpts_new[(cpts_new['Specialty']=='OCCUPATIONAL THERAPY') 
             ##& (refs_results['status_name']=='DENIED - CM')
            & (cpts_new['aa-yn']==1)].tail(10)

,CPT_Code,HCP_CONNECT_AUTH_NUMBER,ref_type,status_name,Specialty,region,UNITS,LOB,is_autoapp,is_PPL,is_app,is_den,auto_approvable,aa-yn


## Referral Detail Lookup (Enter Auth)

In [252]:
cpts_new[(cpts_new['is_den']==1) &
        (cpts_new['aa-yn']==1)].to_csv('../Data/Outputs/false_positives_from_new.csv')

## Auth Lookup (Enter Specialty and Code)

In [253]:
cpts_new[(cpts_new['Specialty']=='ENT-OTOLARYNGOLOGY')
        & (cpts_new['CPT_Code']=='95024')
        ##& (refs_det['auto_approvable'] == 0)
        ]

,CPT_Code,HCP_CONNECT_AUTH_NUMBER,ref_type,status_name,Specialty,region,UNITS,LOB,is_autoapp,is_PPL,is_app,is_den,auto_approvable,aa-yn
94938,95024,15991663H,PHYSICIAN,APPROVED - AUTO,ENT-OTOLARYNGOLOGY,LA/DOWNTOWN,1,_b,1,1,1,0,1.0,1
97510,95024,15948506H,PHYSICIAN,APPROVED - AUTO,ENT-OTOLARYNGOLOGY,LA/DOWNTOWN,1,_s,1,1,1,0,1.0,1
128417,95024,15981397H,PHYSICIAN,APPROVED - AUTO,ENT-OTOLARYNGOLOGY,LA/DOWNTOWN,1,_b,1,1,1,0,1.0,1
137190,95024,15998178H,PHYSICIAN,APPROVED - AUTO,ENT-OTOLARYNGOLOGY,SAN GABRIEL VALLEY,1,_b,1,1,1,0,1.0,1
256466,95024,16139031H,PHYSICIAN,APPROVED - CM,ENT-OTOLARYNGOLOGY,SAN GABRIEL VALLEY,1,_s,0,0,1,0,0.0,0
272335,95024,16154263H,PHYSICIAN,APPROVED - AUTO,ENT-OTOLARYNGOLOGY,LA/DOWNTOWN,1,_b,1,1,1,0,1.0,1
283960,95024,16171368H,PHYSICIAN,APPROVED - AUTO,ENT-OTOLARYNGOLOGY,SAN GABRIEL VALLEY,1,_b,1,1,1,0,1.0,1
382600,95024,16165658H,PHYSICIAN,APPROVED - AUTO,ENT-OTOLARYNGOLOGY,SAN GABRIEL VALLEY,1,_b,1,1,1,0,1.0,1
429609,95024,16216052H,PHYSICIAN,APPROVED - AUTO,ENT-OTOLARYNGOLOGY,SAN GABRIEL VALLEY,1,_b,1,1,1,0,1.0,1
460228,95024,16252444H,PHYSICIAN,APPROVED - AUTO,ENT-OTOLARYNGOLOGY,LA/DOWNTOWN,1,_b,1,1,1,0,1.0,1


In [254]:
cpts_new[(cpts_new['Specialty']=='ENT-OTOLARYNGOLOGY')
        & (cpts_new['HCP_CONNECT_AUTH_NUMBER']=='13418107H')
        ##& (refs_det['auto_approvable'] == 0)
        ]

,CPT_Code,HCP_CONNECT_AUTH_NUMBER,ref_type,status_name,Specialty,region,UNITS,LOB,is_autoapp,is_PPL,is_app,is_den,auto_approvable,aa-yn


In [255]:
# def create_final_statuses(ref_summary, spec_dict_PPL):
#     final_status=[]
#     for index, row in ref_summary.iterrows():
#         if row['is_PPL'] == 1:
#             if row['CPT_Code'] in spec_dict_PPL[row['Specialty']]:
#                 final_status.append(1)
#             else:
#                 final_status.append(0)
#         else:
#             final_status.append(0)
#     return final_status

In [256]:
# final_status = create_final_statuses(cpts_w_claims_fin, spec_dict_PPL)

In [257]:
cpts_w_claims_fin.drop(['cnt_hcp_cost'], axis=1, inplace=True)

In [258]:
cpts_w_claims_fin = cpts_w_claims_fin.merge(cpt_desc, how='inner', on = 'CPT_Code')

In [259]:
cpts_w_claims_fin[cpts_w_claims_fin['Specialty']=='CARDIOLOGY']

,Specialty,CPT_Code,is_PPL,UNITS,is_autoapp,is_den,UNITS_man,cost_to_review,avg_hcp_cost,sum_cost_denied,ROI,fin_aa_rec,dec_source,overrule_rsn,final_decision,aa-yn,Name
209,CARDIOLOGY,96372,0,0,0.000000,0.000000,0.0,0.00,23.540000,0.000000,0.010000,1,clinical,Pend EPL,0.0,NaN,"THER/PROPH/DIAG INJ, SC/I"
210,CARDIOLOGY,96372,1,1,1.000000,0.000000,0.0,NaN,23.540000,0.000000,0.010000,1,Rule,low volume,0.0,0.000000,"THER/PROPH/DIAG INJ, SC/I"
346,CARDIOLOGY,97110,1,2,0.000000,0.000000,2.0,5.06,42.104119,0.000000,0.000000,1,Rule,low volume,0.0,NaN,THERAPEUTIC EXERCISES
572,CARDIOLOGY,97163,1,1,0.000000,0.000000,1.0,2.53,70.760775,0.000000,0.000000,1,Rule,low volume,0.0,NaN,PHYSICAL THERAPY EVALUATI
733,CARDIOLOGY,99201,1,2,0.500000,0.000000,1.0,2.53,39.430000,0.000000,0.000000,1,Rule,low volume,0.0,0.000000,NEW PT OFFICE VISIT-FOCUS
771,CARDIOLOGY,99202,0,1,0.000000,0.000000,1.0,2.53,58.338800,0.000000,0.000000,1,clinical,Pend EPL,0.0,NaN,NEW PT OFFICE VISIT-EXPAN
772,CARDIOLOGY,99202,1,3,0.666667,0.000000,1.0,2.53,58.338800,0.000000,0.000000,1,Rule,low volume,0.0,0.000000,NEW PT OFFICE VISIT-EXPAN
848,CARDIOLOGY,99203,0,2653,0.005648,0.005271,2638.0,6674.14,89.439900,1250.744264,0.187402,1,clinical,Pend EPL,0.0,0.000000,NEW PT OFFICE VISIT-DETAI
849,CARDIOLOGY,99203,1,31569,0.818003,0.000000,5745.0,14534.85,89.439900,0.000000,0.000000,1,DBA,NaN,1.0,0.981182,NEW PT OFFICE VISIT-DETAI
1039,CARDIOLOGY,99204,0,372,0.000000,0.005333,372.0,941.16,138.170800,274.130867,0.291269,1,clinical,Pend EPL,0.0,0.000000,NEW PT OFFICE VISIT-COMPR


In [260]:
# Give the filename you wish to save the file to
cpts_w_claims_fin_filename = '../Data/Outputs/spec_cpt_summary.xlsx'

# Use this function to search for any files which match your filename
files_present = os.path.isfile(cpts_w_claims_fin_filename)

# if no matching files, write to csv, if there are matching files, print statement
if not files_present:
    cpts_w_claims_fin.to_excel(cpts_w_claims_fin_filename, index=False, float_format='%.2f',
                          header=['Specialty', 'CPT Code', 'PPL?', 'volume', 'old rate', 'denial rate',
       'volume, reviewed', 'cost to review', 'avg. cost', 'denied dollars', 'ROI',
       'decision by ROI', 'decision source', 'clinical reason', 'final decision', 'new rate',
       'CPT desc'], 
                          freeze_panes=(1,0))
else:
    print('WARNING: This file already exists!')